# Data Loading

In [18]:
import pandas as pd
import numpy as np

In [19]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

In [20]:
def get_gini(df, label):
    total_instances = len(df)

    # 레이블 값들의 빈도수 계산
    label_counts = df[label].value_counts()

    # 지니 계수 초기화
    gini = 1.0

    # 레이블 값들에 대해 불순도 계산
    for count in label_counts:
        probability = count / total_instances
        gini -= probability ** 2

    return gini

In [21]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [22]:
from itertools import combinations

def get_binary_split(df, attribute):

    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트

    result = []

    # 1부터 속성의 전체 개수 -1 까지의 길이를 가진 조합 만들기
    for i in range(1, len(uniques)):
        # combinations 함수를 사용하여 조합을 생성
        splits = list(combinations(uniques, i))
        result.extend(splits)

    return result

In [23]:
get_binary_split(pd_data,'income')

[('high',),
 ('medium',),
 ('low',),
 ('high', 'medium'),
 ('high', 'low'),
 ('medium', 'low')]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [24]:
def get_attribute_gini_index(df, attribute, label):
    result = {}
    binary_split = get_binary_split(df, attribute)
    n = len(df)

    for split in binary_split:
       # 각 이진 분할의 이름 생성 (소문자로 변경)
        col_name = "_".join(split).lower()

        # 이진 분할에 따른 마스크 생성
        mask = np.array(list(map(lambda x: x in split, df[attribute])))
        # 각 분할에 해당하는 데이터 추출
        data_t = df.loc[mask, :]
        data_r = df.loc[~mask, :]

        # 각 분할에 대한 지니 인덱스 계산
        gini_t = get_gini(data_t, label)
        gini_r = get_gini(data_r, label)

        # 전체 지니 인덱스 계산 및 결과에 추가
        gini = len(data_t) / n * gini_t + len(data_r) / n * gini_r
        result[col_name] = gini

    return result

In [25]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{'high': 0.4428571428571429,
 'medium': 0.4583333333333333,
 'low': 0.45,
 'high_medium': 0.45,
 'high_low': 0.4583333333333333,
 'medium_low': 0.4428571428571429}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [26]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

('high', 0.4428571428571429)

In [27]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

'high'

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [28]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    # 이진 분할과 지니 인덱스 계산
    gini_index_result = get_attribute_gini_index(pd_data, feature, label)

    # 가장 낮은 Gini 계수와 변수 출력
    min_gini_feature = min(gini_index_result, key=gini_index_result.get)
    min_gini_index = gini_index_result[min_gini_feature]
    print(f"Minimum Gini Index of {feature} : {min_gini_index}")

Minimum Gini Index of age : 0.35714285714285715
Minimum Gini Index of income : 0.4428571428571429
Minimum Gini Index of student : 0.3673469387755103
Minimum Gini Index of credit_rating : 0.42857142857142855


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [29]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{'youth': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'senior': 0.4571428571428572,
 'youth_middle_aged': 0.4571428571428572,
 'youth_senior': 0.35714285714285715,
 'middle_aged_senior': 0.3936507936507937}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [30]:
from math import log2

def getEntropy(df, feature):
    total_instances = len(df)
    entropy = 0.0

    # feature 열의 고유값과 각각의 빈도수를 가져옴
    value_counts = df[feature].value_counts()

    # 엔트로피 계산
    for count in value_counts:
        probability = count / total_instances
        entropy -= probability * log2(probability)

    return entropy

In [31]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [32]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature):
    result = {}

  # 목표 변수의 엔트로피 계산
    target_entropy = getEntropy(df, feature)
    total_instances = len(df)

    # 목표 변수를 제외한 각 변수에 대해 정보 획득량 계산
    for column in df.columns:
        if column != feature:
            column_entropy = 0
            value_counts = df[column].value_counts()

            # 각 변수 값에 대한 정보 획득량 계산
            for value, count in value_counts.items():
                subset = df[df[column] == value]
                subset_entropy = getEntropy(subset, feature)
                column_entropy += (count / total_instances) * subset_entropy

            # 각 변수의 정보 획득량을 결과에 추가
            information_gain = target_entropy - column_entropy
            result[column] = information_gain
    return result

In [33]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}